# Vamos agora falar de interação 🕹️

fonte e material de apoio : https://www.kaggle.com/discussions/getting-started/100831


## Os recursos de visualização podem ser muito bem aproveitados se inserirmos recursos onde o usuário pode interagir com nossa visualização 📊

### Para isso temos, algumas opções : ipywidgets, D3.js, Bokeh, Plotly , Dash.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd


/tmp/ipykernel_44521/2402724060.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_44521/2402724060.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
df = pd.read_excel("https://github.com/chris1610/pbpython/blob/master/data/salesfunnel.xlsx?raw=True")
pv = pd.pivot_table(df, index=['Name'], columns=["Status"], values=['Quantity'], aggfunc=sum, fill_value=0)

In [3]:
trace1 = go.Bar(x=pv.index, y=pv[('Quantity', 'declined')], name='Declined')
trace2 = go.Bar(x=pv.index, y=pv[('Quantity', 'pending')], name='Pending')
trace3 = go.Bar(x=pv.index, y=pv[('Quantity', 'presented')], name='Presented')
trace4 = go.Bar(x=pv.index, y=pv[('Quantity', 'won')], name='Won')

### Primeiro vamos ao layout
### Para fazer isso utilizaremos componentes HTML que já conhecemos, como H1, Divs, Tabs, 

In [4]:
app1 = dash.Dash()

app1.layout = html.Div([
    html.H1('Sales Funnel Report'),
    html.Div('National Sales Funnel Report.'),
    dcc.Graph(
        id='example-graph',
        figure={
            'data': [trace1, trace2, trace3, trace4],
            'layout':
            go.Layout(title='Order Status by Customer', barmode='stack')
        })
])

In [6]:
app1.run_server(debug=True)

## Vamos agora criar uma interação, colocando a opção para o usuário inserir o 'Manager' que ele quer visualizar.
## Faremos isso com um dropdown
## Nele, colocaremos as opções de 'managers' que o usuário poderá selecionar

In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd

df = pd.read_excel(
    "https://github.com/chris1610/pbpython/blob/master/data/salesfunnel.xlsx?raw=True"
)
mgr_options = list(df["Manager"].unique())



mgr_options.append('All Managers')

app = dash.Dash()

app.layout = html.Div([
    html.H2("Sales Funnel Report"),
    html.Div(                        # Criaremos outra div
        [   
            dcc.Dropdown(            # E dentro dela um dropdown
                id="Manager",
                options=[{
                    'label': i,
                    'value': i
                } for i in mgr_options],
                value='All Managers'),
        ],
        style={'width': '25%',              # podemos adicionar CSS desta forma
               'display': 'inline-block'}),
    dcc.Graph(id='funnel-graph'),
])


@app.callback(
    dash.dependencies.Output('funnel-graph', 'figure'),
    [dash.dependencies.Input('Manager', 'value')])
def update_graph(Manager):
    if Manager == "All Managers":
        df_plot = df.copy()
    else:
        df_plot = df[df['Manager'] == Manager]

    pv = pd.pivot_table(
        df_plot,
        index=['Name'],
        columns=["Status"],
        values=['Quantity'],
        aggfunc=sum,
        fill_value=0)

    trace1 = go.Bar(x=pv.index, y=pv[('Quantity', 'declined')], name='Declined')
    trace2 = go.Bar(x=pv.index, y=pv[('Quantity', 'pending')], name='Pending')
    trace3 = go.Bar(x=pv.index, y=pv[('Quantity', 'presented')], name='Presented')
    trace4 = go.Bar(x=pv.index, y=pv[('Quantity', 'won')], name='Won')

    return {
        'data': [trace1, trace2, trace3, trace4],
        'layout':
        go.Layout(
            title='Customer Order Status for {}'.format(Manager),
            barmode='stack')
    }


if __name__ == '__main__':
    app.run_server(debug=True)

## Muito interessante as possibilidades correto? Vamos entender um pouco do funcionamento básico dessa biblioteca

## A primeira parte é sobre o layout e depois falaremos sobre as callbacks

### Temos abaixo um código simples de um dataset sobre expectativa de vida no mundo.

In [8]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.


from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd


app = Dash(__name__)

df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 log_x=True, size_max=60)

app.layout = html.Div([
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run(debug=True)

### Porém ele está muito simples, vamos colocar mais plots nele..

In [9]:
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# Carregue os dados do DataFrame
df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

# Crie a primeira visualização de gráfico (gráfico de dispersão)
fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 log_x=True, size_max=60)

# Crie uma segunda visualização de gráfico (por exemplo, um gráfico de barras)
# Substitua os dados e a configuração de gráfico apropriados para a segunda visualização
fig2 = px.bar(df, x="country", y="gdp per capita")

app.layout = html.Div([
    dcc.Graph(
        id='life-exp-vs-gdp',
        figure=fig
    ),
    dcc.Graph(
        id='gdp-per-capita-bar',
        figure=fig2  # Use a segunda visualização de gráfico aqui
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


### Que tal deixar ele mais bonito e organizado? CSS nele.

In [10]:
from dash import Dash, dcc, html
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# Load the data into a DataFrame
df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

# Create the first visualization (scatter plot)
fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 log_x=True, size_max=60)

# Create the second visualization (bar chart)
fig2 = px.bar(df, x="country", y="gdp per capita")

app.layout = html.Div([
    html.Div([
        html.H3("Scatter Plot: Life Expectancy vs. GDP Per Capita"),
        dcc.Graph(
            id='life-exp-vs-gdp',
            figure=fig
        ),
    ], style={'width': '49%', 'display': 'inline-block'}),
    
    html.Div([
        html.H3("Bar Chart: GDP Per Capita by Country"),
        dcc.Graph(
            id='gdp-per-capita-bar',
            figure=fig2
        ),
    ], style={'width': '49%', 'display': 'inline-block'})
])

if __name__ == '__main__':
    app.run_server(debug=True)


## E agora funções de interação de dados.

In [11]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

app = Dash(__name__)

# Load the data into a DataFrame
df = pd.read_csv('https://gist.githubusercontent.com/chriddyp/5d1ea79569ed194d432e56108a04d188/raw/a9f9e8076b837d541398e999dcbac2b2826a81f8/gdp-life-exp-2007.csv')

# Create the first visualization (scatter plot)
fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                 size="population", color="continent", hover_name="country",
                 size_max=60)

# Create the second visualization (bar chart)
fig2 = px.bar(df, x="country", y="gdp per capita")

app.layout = html.Div([
    html.Div([
        html.H3("Scatter Plot: Life Expectancy vs. GDP Per Capita"),
        dcc.RadioItems(
            id='log-scale',
            options=[
                {'label': 'Log Scale', 'value': 'log'},
                {'label': 'Linear Scale', 'value': 'linear'}
            ],
            value='log',
            labelStyle={'display': 'block'}
        ),
        dcc.Graph(
            id='life-exp-vs-gdp',
            figure=fig
        )
    ], style={'width': '49%', 'display': 'inline-block'}),
    
    html.Div([
        html.H3("Bar Chart: GDP Per Capita by Country"),
        dcc.Graph(
            id='gdp-per-capita-bar',
            figure=fig2
        ),
        dcc.Slider(
            id='country-count-slider',
            min=10,
            max=len(df),
            step=10,
            value=len(df),
            marks={i: {'label': str(i), 'style': {'transform': 'rotate(-45deg)'}} for i in range(10, len(df) + 1, 10)},
        ),
        dcc.Dropdown(
            id='sorting-dropdown',
            options=[
                {'label': 'Sort by Alphabetical Order', 'value': 'alphabetical'},
                {'label': 'Sort by GDP per Capita', 'value': 'gdp'},
            ],
            value='alphabetical',
            style={'width': '50%'}
        )
    ], style={'width': '49%', 'display': 'inline-block'})
])

@app.callback(
    Output('gdp-per-capita-bar', 'figure'),
    Input('country-count-slider', 'value'),
    Input('sorting-dropdown', 'value')
)
def update_country_count(selected_count, sorting_option):
    if sorting_option == 'alphabetical':
        sorted_df = df.sort_values(by='country').head(selected_count)
    else:
        sorted_df = df.sort_values(by='gdp per capita', ascending=False).head(selected_count)
    
    updated_fig = px.bar(sorted_df, x="country", y="gdp per capita")
    return updated_fig

@app.callback(
    Output('life-exp-vs-gdp', 'figure'),
    Input('log-scale', 'value')
)
def update_scale(selected_scale):
    updated_fig = px.scatter(df, x="gdp per capita", y="life expectancy",
                             size="population", color="continent", hover_name="country",
                             size_max=60, log_x=(selected_scale == 'log'))
    return updated_fig

if __name__ == '__main__':
    app.run_server(debug=True)


: 